In [1]:
import numpy as np
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from skimage import measure
import pyrender
import torch
from transformers import AutoTokenizer, AutoModel
import os


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def select_image():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    return file_path

In [3]:
def generate_3d_volume(image_path, depth_layers=30):
    image = cv2.imread(image_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    norm = gray.astype(float) / 255.0
    
    volume = np.zeros((gray.shape[0], gray.shape[1], depth_layers))
    for z in range(depth_layers):
        factor = 1.0 - (z / depth_layers)
        volume[:, :, z] = norm * factor
    return volume, rgb


In [4]:
def create_mesh(volume, rgb):
    verts, faces, _, _ = measure.marching_cubes(volume, level=0.1)
    mesh = o3d.geometry.TriangleMesh()
    
    verts_swapped = np.column_stack([verts[:,1], verts[:,0], verts[:,2]])
    mesh.vertices = o3d.utility.Vector3dVector(verts_swapped)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    
    h, w = rgb.shape[:2]
    vertex_colors = []
    
    for vertex in verts_swapped:
        x = int(np.clip(vertex[0], 0, w - 1))
        y = int(np.clip(vertex[1], 0, h - 1))
        color = rgb[y, x] / 255.0
        vertex_colors.append(color)
    
    mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
    mesh.compute_vertex_normals()
    return mesh


In [5]:
def save_and_render(mesh, output_file="output_model.stl"):
    o3d.io.write_triangle_mesh(output_file, mesh)
    print(f"Mesh saved as: {output_file}")
    
    pyrender_mesh = pyrender.Mesh.from_trimesh(mesh)
    scene = pyrender.Scene()
    scene.add(pyrender_mesh)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True)


In [ ]:
image_path = select_image()

if image_path:
    volume, rgb = generate_3d_volume(image_path)
    mesh = create_mesh(volume, rgb)
    save_and_render(mesh)
else:
    print("No image selected.")
